# Let's generate some text

No training here, only fun stuff.

In [1]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en')

In [5]:
DATA_PATH=Path('data/')
PATH = DATA_PATH/'aclImdb'

BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

Note that the itos/stoi have to match the model you're loading! Obvious but, you know. Remember.

In [23]:
itos_name = 'itos_imdb.pkl'  # 'itos_wt103.pkl'
model_name = 'lm_last_ft'    # 'pretrained'

itos2 = pickle.load((PRE_PATH/itos_name).open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

trn_lm = np.array([[0]])
val_lm = np.array([[0]])

vs=len(itos2)

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.load(model_name)
m=learner.model

In [24]:
def generate_text(m, s, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi2[w] for w in ss]
    t = torch.autograd.Variable(torch.LongTensor(np.array([si])))
    
    res,*_ = m(t)

    print(s,"\n")
    for i in range(l):
        n = res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        print(itos2[n], end=' ')
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))
    print('...')

In [26]:
generate_text(m, "the movie was", l=50)

the movie was 

a few days later , the first of the three , the " the man with the golden gun " , was released . the first of these was the " the man who killed the man " , which was released on the same day as the first . ...
